# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
#Import weather data from part I
csv_path = os.path.join("..", "output_data", "clean_city_data.csv")

clean_city_data_df = pd.read_csv(csv_path)
clean_city_data_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date
0,marzuq,23.05727,15.03735,82.27,33,0,0.89,ly,1600315525
1,victoria,-4.00996,57.14193,87.01,82,94,1.99,sc,1600315302
2,saint anthony,55.13695,-55.13695,57.20,66,1,3.36,ca,1600315525
3,auki,-6.01494,163.40587,85.42,71,100,8.03,sb,1600315525
4,kaeo,-26.06474,176.43824,66.99,48,0,8.99,nz,1600315525
...,...,...,...,...,...,...,...,...,...
539,manokwari,0.00000,133.33117,84.27,66,22,4.18,id,1600315369
540,turukhansk,64.15936,88.21912,51.91,83,96,14.90,ru,1600315652
541,zachary,31.07719,-91.22659,77.00,88,1,3.36,us,1600315652
542,tabas,33.08217,56.13944,71.60,30,0,4.70,ir,1600315652


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [67]:
# SET UP LAT LONG PAIRS
Coords1 = clean_city_data_df[['Latitude', 'Longitude']]

# Set up a list of humidities
Humidity = clean_city_data_df['Humidity']
Humidity

# CONFIGURE GMAPS
gmaps.configure(api_key=g_key)

# Customize Layout
figure_layout = {
    'width': '800px',
    'height': '600px'
}

# store the figure in a variable so we can continue to add layers to the same map
Myfig = gmaps.figure(layout=figure_layout, center=(0, 0), zoom_level=2)

# Create a marker layer using our coordinates
heat_layer = gmaps.heatmap_layer(Coords1, weights=Humidity)

# Add the layer to the map
Myfig.add_layer(heat_layer)

# Save and display the figure with the newly added layer
Myfig


Figure(layout=FigureLayout(height='600px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#  Selecting ideal cities. Temp <85, >65, Wind speed <15, Cloudiness <10, Humidity<55
ideal_weather_cities_df = clean_city_data_df.loc[(clean_city_data_df["Max_Temp"]<85) & (clean_city_data_df["Max_Temp"]>65) & 
                                                        (clean_city_data_df["Wind_Speed"]<15) & (clean_city_data_df["Cloudiness"]<10) & 
                                                        (clean_city_data_df["Humidity"]<55) & (clean_city_data_df["Humidity"]>35) ]

print(f"No. of ideal cities: {ideal_weather_cities_df['City'].count()}")
ideal_weather_cities_df

No. of ideal cities: 12


,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date
4,kaeo,-26.06474,176.43824,66.99,48,0,8.99,nz,1600315525
65,shiraz,30.07470,52.12948,66.20,37,0,2.57,ir,1600315539
79,hwange,-19.04731,26.06474,66.61,37,0,2.24,zw,1600315542
135,san quintin,21.05229,-128.31872,79.63,46,0,10.56,mx,1600315555
177,guerrero negro,16.03984,-129.32121,79.45,52,6,10.60,mx,1600315564
207,zvishavane,-20.04980,30.07470,66.65,43,2,1.16,zw,1600315571
208,zhangjiakou,42.10458,114.28386,65.03,37,0,13.82,cn,1600315571
257,gunnedah,-31.07719,150.37350,77.00,38,0,5.82,au,1600315582
363,morros,-10.02490,-46.11454,72.19,52,0,3.33,br,1600315607
393,laguna,-36.08964,-39.09711,77.00,50,1,6.93,br,1600315613


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# Create new hotel df, and create extra columns
hotel_df = ideal_weather_cities_df.copy()
hotel_df['Country'] = hotel_df['Country_Code']
hotel_df["Hotel Name"] = ""
hotel_df["Hotel_Lat"] = ""
hotel_df["Hotel_Long"] = ""

hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date,Country,Hotel Name,Hotel_Lat,Hotel_Long
4,kaeo,-26.06474,176.43824,66.99,48,0,8.99,nz,1600315525,nz,,,
65,shiraz,30.07470,52.12948,66.20,37,0,2.57,ir,1600315539,ir,,,
79,hwange,-19.04731,26.06474,66.61,37,0,2.24,zw,1600315542,zw,,,
135,san quintin,21.05229,-128.31872,79.63,46,0,10.56,mx,1600315555,mx,,,
177,guerrero negro,16.03984,-129.32121,79.45,52,6,10.60,mx,1600315564,mx,,,
207,zvishavane,-20.04980,30.07470,66.65,43,2,1.16,zw,1600315571,zw,,,
208,zhangjiakou,42.10458,114.28386,65.03,37,0,13.82,cn,1600315571,cn,,,
257,gunnedah,-31.07719,150.37350,77.00,38,0,5.82,au,1600315582,au,,,
363,morros,-10.02490,-46.11454,72.19,52,0,3.33,br,1600315607,br,,,
393,laguna,-36.08964,-39.09711,77.00,50,1,6.93,br,1600315613,br,,,


In [42]:
# geocoordinates for the base of our search
target_location = "29.714033,-95.406480" # coordinates for Rice University

# distance, IN METERS, within which the place results must live
target_radius = 5000

# keyword term to be matched against all of Google's indexed content
target_keyword = "Hotel"

# type of establishment that we'd like to use to filter our place results
target_type = "Hotel"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a dictionary to store all query parameters
parameters = {
#     "location": target_location,
    "keyword": target_keyword,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# # # run a request using our params dictionary
# run a request using our params dictionary
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    print(f"Searching for hotel in city with lat lng {lat}, {lng}...")
# update params dictionary
    parameters['location'] = f"{lat},{lng}"
    hotel_info = requests.get(base_url, params=parameters)
    hotel_info_json = hotel_info.json()
# Incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] =  hotel_info_json['results'][0]['name']
        hotel_df.loc[index, 'Hotel_Lat'] =  hotel_info_json['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel_Long'] =  hotel_info_json['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
print(f"No. of Hotels found in our ideal cities: {hotel_df['City'].count()}")
hotel_df

Searching for hotel in city with lat lng -26.064740000000004, 176.43823999999998...
Missing field/result... skipping.
Searching for hotel in city with lat lng 30.074700000000004, 52.12948000000001...
Searching for hotel in city with lat lng -19.047310000000003, 26.064740000000004...
Missing field/result... skipping.
Searching for hotel in city with lat lng 21.052290000000006, -128.31871999999998...
Missing field/result... skipping.
Searching for hotel in city with lat lng 16.039839999999998, -129.32121...
Missing field/result... skipping.
Searching for hotel in city with lat lng -20.0498, 30.074700000000004...
Missing field/result... skipping.
Searching for hotel in city with lat lng 42.10458000000001, 114.28386000000002...
Missing field/result... skipping.
Searching for hotel in city with lat lng -31.07719, 150.3735...
Missing field/result... skipping.
Searching for hotel in city with lat lng -10.0249, -46.11454000000001...
Missing field/result... skipping.
Searching for hotel in city

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date,Country,Hotel Name,Hotel_Lat,Hotel_Long
4,kaeo,-26.06474,176.43824,66.99,48,0,8.99,nz,1600315525,nz,,,
65,shiraz,30.07470,52.12948,66.20,37,0,2.57,ir,1600315539,ir,boyaq Ecolodge,30.0449,52.1283
79,hwange,-19.04731,26.06474,66.61,37,0,2.24,zw,1600315542,zw,Nantwich Lodge,-18.598,25.9042
135,san quintin,21.05229,-128.31872,79.63,46,0,10.56,mx,1600315555,mx,,,
177,guerrero negro,16.03984,-129.32121,79.45,52,6,10.60,mx,1600315564,mx,,,
207,zvishavane,-20.04980,30.07470,66.65,43,2,1.16,zw,1600315571,zw,Hotel Boutique Runde,-20.3138,30.0581
208,zhangjiakou,42.10458,114.28386,65.03,37,0,13.82,cn,1600315571,cn,Ramada Encore by Wyndham Chongli Secret Garden...,41.7433,114.386
257,gunnedah,-31.07719,150.37350,77.00,38,0,5.82,au,1600315582,au,Best Western Quirindi RSL Motel,-31.5026,150.679
363,morros,-10.02490,-46.11454,72.19,52,0,3.33,br,1600315607,br,Pousada Rio Novo do Jalapao,-10.3854,-46.4393
393,laguna,-36.08964,-39.09711,77.00,50,1,6.93,br,1600315613,br,,,


In [43]:
# Clean dataframe
# hotel_df.dropna(subset=['Hotel Name'], inplace=True)
# hotel_df = hotel_df['Hotel Name'].dropna(inplace=True)

hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Code,Date,Country,Hotel Name,Hotel_Lat,Hotel_Long
4,kaeo,-26.06474,176.43824,66.99,48,0,8.99,nz,1600315525,nz,,,
65,shiraz,30.07470,52.12948,66.20,37,0,2.57,ir,1600315539,ir,boyaq Ecolodge,30.0449,52.1283
79,hwange,-19.04731,26.06474,66.61,37,0,2.24,zw,1600315542,zw,Nantwich Lodge,-18.598,25.9042
135,san quintin,21.05229,-128.31872,79.63,46,0,10.56,mx,1600315555,mx,,,
177,guerrero negro,16.03984,-129.32121,79.45,52,6,10.60,mx,1600315564,mx,,,
207,zvishavane,-20.04980,30.07470,66.65,43,2,1.16,zw,1600315571,zw,Hotel Boutique Runde,-20.3138,30.0581
208,zhangjiakou,42.10458,114.28386,65.03,37,0,13.82,cn,1600315571,cn,Ramada Encore by Wyndham Chongli Secret Garden...,41.7433,114.386
257,gunnedah,-31.07719,150.37350,77.00,38,0,5.82,au,1600315582,au,Best Western Quirindi RSL Motel,-31.5026,150.679
363,morros,-10.02490,-46.11454,72.19,52,0,3.33,br,1600315607,br,Pousada Rio Novo do Jalapao,-10.3854,-46.4393
393,laguna,-36.08964,-39.09711,77.00,50,1,6.93,br,1600315613,br,,,


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [68]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations)
Myfig.add_layer(hotel_layer)

# Save and display figure
Myfig

Figure(layout=FigureLayout(height='600px', width='800px'))